In [7]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel,get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [2]:
## load network
net = torchvision.models.resnet50(pretrained=True)
#net = torchvision.models.densenet121(pretrained=True)
#net = torchvision.models.inception_v3(pretrained=True) #Unfortunately, inception_v3 doesn't work now, sorry
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [3]:
## image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [4]:
## create save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_complexCNN_conv'
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [5]:
# initial image for the optimization
## note that the input shape of inception_v3 model is (229,229,3), not (224,224,3)
h, w = 299,299 # for inception_v3
h,w = 224, 224
initial_input = np.random.randint(0, 256, (h,w,3))

In [6]:
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
## set target layer 
#target_layer = "Conv2d_2a_3x3.conv"
#target_layer = "fc"
target_layer = "layer3[4].conv3"
target_layer_list = [target_layer]

In [9]:
# options
opts = {
    'img_mean': img_mean, #img_mean to preprocessing input image
    'img_std': img_std,   #img_std to preprocessing input image
    'norm': norm,         #if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
    
    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start':1, # learning rate
    'lr_end': 1,   # we can change learning rate linearly setteing these two parameters

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,   # we can also change pixel decay linealy 

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during optimization
    'jitter_size': 32,
    
    'use_p_norm_reg': False, # use p_norm regularization
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    'input_size': (h,w,3) # you can set only input shape instead of initial_input image. in this case, intial_input  
                           # is 0-255 random image whose shape matches to input_size
    }



In [10]:
channel_list = [14,56]

In [11]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # generate preferred image
    preferred_stim = generate_preferred(net, target_layer_list, channel, **opts)
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=0.054995;
iter=2; mean(abs(feat))=0.0093337;
iter=3; mean(abs(feat))=0.0298691;
iter=4; mean(abs(feat))=0.0047677;
iter=5; mean(abs(feat))=0.00551627;
iter=6; mean(abs(feat))=0.0299947;
iter=7; mean(abs(feat))=0.019228;
iter=8; mean(abs(feat))=0.00955165;
iter=9; mean(abs(feat))=0.0285365;
iter=10; mean(abs(feat))=0.0376949;
iter=11; mean(abs(feat))=0.0202187;
iter=12; mean(abs(feat))=0.0266327;
iter=13; mean(abs(feat))=0.00865701;
iter=14; mean(abs(feat))=0.0200127;
iter=15; mean(abs(feat))=0.0124109;
iter=16; mean(abs(feat))=0.0214793;
iter=17; mean(abs(feat))=0.0256208;
iter=18; mean(abs(feat))=0.0223351;
iter=19; mean(abs(feat))=0.0142123;
iter=20; mean(abs(feat))=0.0492294;
iter=21; mean(abs(feat))=0.0177306;
iter=22; mean(abs(feat))=0.0218978;
iter=23; mean(abs(feat))=0.0189857;
iter=24; mean(abs(feat))=0.0349643;
iter=25; mean(abs(feat))=0.0438188;
iter=26; mean(abs(feat))=0.0446551;
iter=27; mean(abs(feat))=0.0101511;
iter=28; mean(abs(feat)

iter=26; mean(abs(feat))=0.00968736;
iter=27; mean(abs(feat))=0.0039482;
iter=28; mean(abs(feat))=0.0142139;
iter=29; mean(abs(feat))=0.00542582;
iter=30; mean(abs(feat))=0.00610022;
iter=31; mean(abs(feat))=0.00410004;
iter=32; mean(abs(feat))=0.0123987;
iter=33; mean(abs(feat))=0.0132824;
iter=34; mean(abs(feat))=0.00704486;
iter=35; mean(abs(feat))=0.00670963;
iter=36; mean(abs(feat))=0.0030494;
iter=37; mean(abs(feat))=0.00751322;
iter=38; mean(abs(feat))=0.000776885;
iter=39; mean(abs(feat))=0.0264886;
iter=40; mean(abs(feat))=0.0242469;
iter=41; mean(abs(feat))=0.00733269;
iter=42; mean(abs(feat))=0.00134256;
iter=43; mean(abs(feat))=0.00386397;
iter=44; mean(abs(feat))=0.0229923;
iter=45; mean(abs(feat))=0.0264552;
iter=46; mean(abs(feat))=0.0194154;
iter=47; mean(abs(feat))=0.00543439;
iter=48; mean(abs(feat))=0.0172202;
iter=49; mean(abs(feat))=0.0202114;
iter=50; mean(abs(feat))=0.0174456;
iter=51; mean(abs(feat))=0.0285891;
iter=52; mean(abs(feat))=0.0118295;
iter=53; mean(a